In [1]:
from sklearn.tree._tree import TREE_LEAF
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score, mean_squared_error, mean_absolute_error
import copy as copy
import importlib
import numpy as numpy
from statistics import mean
import os
import pickle
import pydotplus

import functions as func

labels_class = ["Prog1_scaled","Prog1_beste", "MfI1_beste", "beste", "beide"]
others_class = [["Prog1_beste","MfI1_beste","beste","beide"],["MfI1_beste","beste","beide","Prog1_scaled"],["Prog1_beste","beste","beide","Prog1_scaled"],["Prog1_beste","MfI1_beste","beide","Prog1_scaled"]
         ,["Prog1_beste","MfI1_beste","beste","Prog1_scaled"]]
labels_reg = ["Prog1_scaled","Prog1_beste", "MfI1_beste", "beste"]
others_reg = [["Prog1_beste","MfI1_beste","beste"],["MfI1_beste","beste","Prog1_scaled"],["Prog1_beste","beste","Prog1_scaled"],["Prog1_beste","MfI1_beste","Prog1_scaled"]]

measure_class = ["accuracy","f1","roc_auc"]
measure_reg = ["reg_accuracy","mse","mae"]




In [2]:
other_count = 0
#Scenarios 4. - 7.
for LABEL in labels_class:
    print("====================%s=============="%(LABEL))
    #Continuuos labels after normalization and scaling not suitable for accuracy
    if LABEL == "Prog1_scaled":
        other_count+=1
        continue
    for measure in measure_class:
        print("====================%s=============="%(measure))
        
        if LABEL == "beide" and measure != "accuracy":
            continue
        dt = DecisionTreeClassifier(random_state=42)
        data_raw,train_raw,test_raw,validation_raw,names,feature_names = func.load_data_bestanden_raw(LABEL,others_class[other_count][0],
                                                                         others_class[other_count][1],
                                                                         others_class[other_count][2])
        dt.fit(train_raw[:,:-1],train_raw[:,-1])
        #Results before pruning
        func.print_results_class(dt, train_raw, test_raw, validation_raw, LABEL, "vor")
        #Prune
        best_tree, best_score, best_t = func.prune_classifier(dt,measure, validation_raw)
    
        print(best_score)
        print(best_t)
        #Render trees
        os.makedirs(os.path.dirname('graphs/dot/%s/cvp/prune_full_dt.dot'%(LABEL)), exist_ok = True)
        os.makedirs(os.path.dirname('graphs/%s/cvp/prune_full_dt.png'%(LABEL)), exist_ok = True)

        tree.export_graphviz(dt, out_file='graphs/dot/%s/cvp/prune_full_dt.dot'%(LABEL), feature_names= names[:-1],
                         filled=True, rounded=True,special_characters=True)
        os.system("dot -T png graphs/dot/%s/cvp/prune_full_dt.dot -o graphs/%s/cvp/prune_full_dt.png"%(LABEL,LABEL))

        
        os.makedirs(os.path.dirname('graphs/dot/%s/cvp/%s/best_pruned.dot'%(LABEL,measure)), exist_ok = True)
        os.makedirs(os.path.dirname('graphs/%s/cvp/%s/best_pruned.png'%(LABEL,measure)), exist_ok = True)

        tree.export_graphviz(best_tree, out_file='graphs/dot/%s/cvp/%s/best_pruned.dot'%(LABEL,measure), feature_names= names[:-1],
                        filled=True, rounded=True,special_characters=True)
        os.system("dot -T png graphs/dot/%s/cvp/%s/best_pruned.dot -o graphs/%s/cvp/%s/best_pruned.png"%(LABEL,measure,LABEL,measure))
        #Results after pruning
        func.print_results_class(best_tree, train_raw, test_raw, validation_raw, LABEL, "nach")
        #Store model permanently
        os.makedirs(os.path.dirname("pruned_models/CVP/%s/%s/%s_%s.pkl"%(LABEL,measure,LABEL,measure)), exist_ok = True)
        temp_pkl_file = open("pruned_models/CVP/%s/%s/%s_%s.pkl"%(LABEL,measure,LABEL,measure), "wb")
        pickle.dump(copy.deepcopy(best_tree), temp_pkl_file)
        temp_pkl_file.close()
    other_count += 1

====================Prog1_scaled==============
====================Prog1_beste==============
====================accuracy==============
[[83  0]
 [ 0 37]]
ACC vor Pruning Training:  1.0
AUC vor Pruning Training:  1.0
F1 vor Pruning Training:  1.0
[[60 36]
 [17 19]]
ACC vor Pruning Test:  0.5984848484848485
AUC vor Pruning Test:  0.5763888888888888
F1 vor Pruning Test:  0.4175824175824176
[[27 20]
 [ 5  5]]
ACC vor Pruning Validation:  0.5614035087719298
AUC vor Pruning Validation:  0.5372340425531915
F1 vor Pruning Validation:  0.28571428571428575
0.5964912280701754
0.24999999999999933
[[80  3]
 [ 7 30]]
ACC nach Pruning Training:  0.9166666666666666
AUC nach Pruning Training:  0.9817648974275481
F1 nach Pruning Training:  0.8571428571428571
[[66 30]
 [24 12]]
ACC nach Pruning Test:  0.5909090909090909
AUC nach Pruning Test:  0.5711805555555556
F1 nach Pruning Test:  0.30769230769230765
[[30 17]
 [ 6  4]]
ACC nach Pruning Validation:  0.5964912280701754
AUC nach Pruning Validation:  0.

[[99  2]
 [ 4 22]]
ACC nach Pruning Training:  0.952755905511811
AUC nach Pruning Training:  0.9891469916222391
F1 nach Pruning Training:  0.8799999999999999
[[79 14]
 [32  9]]
ACC nach Pruning Test:  0.6567164179104478
AUC nach Pruning Test:  0.5213742460005245
F1 nach Pruning Test:  0.28125
[[21 16]
 [15  5]]
ACC nach Pruning Validation:  0.45614035087719296
AUC nach Pruning Validation:  0.41283783783783784
F1 nach Pruning Validation:  0.24390243902439024
====================beide==============
====================accuracy==============
[[20  0  0]
 [ 0 12  0]
 [ 0  0 37]]
ACC vor Pruning Training:  1.0
[[11  6  5]
 [ 5  2 10]
 [10 10 20]]
ACC vor Pruning Test:  0.4177215189873418
[[11  0  3]
 [ 6  1  3]
 [ 5  0  3]]
ACC vor Pruning Validation:  0.46875
0.46875
0.49999999999999956
[[20  0  0]
 [ 2 10  0]
 [ 2  0 35]]
ACC nach Pruning Training:  0.9420289855072463
[[11  6  5]
 [ 6  1 10]
 [12 10 18]]
ACC nach Pruning Test:  0.379746835443038
[[11  0  3]
 [ 9  1  0]
 [ 5  0  3]]
ACC na

In [3]:
other_count = 0
#Scenarios 1. - 3.
for LABEL in labels_reg:
    print("====================%s=============="%(LABEL))

    for measure in measure_reg:
        print("====================%s=============="%(measure))    
        
        if measure=="reg_accuracy":
            #Continuuos labels after normalization and scaling not suitable for accuracy
            if LABEL == "Prog1_scaled":
                continue
            dt = DecisionTreeClassifier(random_state=42)
            data_raw,train_raw,test_raw,validation_raw,names,feature_names = func.load_data_noten_raw(LABEL,others_class[other_count][0],
                                                                             others_class[other_count][1],
                                                                             others_reg[other_count][2])
            dt.fit(train_raw[:,:-1],train_raw[:,-1])
            #Results before pruning
            func.print_results_class(dt, train_raw, test_raw, validation_raw, LABEL, "vor",noten=True)
            #Prune
            best_tree, best_score, best_t = func.prune_classifier(dt, measure, validation_raw)
            
            
        else:
            dt = DecisionTreeRegressor(random_state=42)
            data_raw,train_raw,test_raw,validation_raw,names,feature_names = func.load_data_noten_raw(LABEL,others_class[other_count][0],
                                                                         others_class[other_count][1],
                                                                         others_reg[other_count][2])
            dt.fit(train_raw[:,:-1],train_raw[:,-1])
            #Results after pruning
            func.print_results_reg(dt, train_raw, test_raw, validation_raw, "vor")
            #Prune
            best_tree, best_score, best_t = func.prune_regressor(dt, measure, validation_raw)
            
        print(best_score)
        print(best_t)
        #Render trees
        os.makedirs(os.path.dirname('graphs/dot/%s/cvp/prune_full_dt_%s.dot'%(LABEL,measure)), exist_ok = True)
        os.makedirs(os.path.dirname('graphs/%s/cvp/prune_full_dt_%s.png'%(LABEL,measure)), exist_ok = True)

        tree.export_graphviz(dt, out_file='graphs/dot/%s/cvp/prune_full_dt_%s.dot'%(LABEL,measure), feature_names= names[:-1],
                         filled=True, rounded=True,special_characters=True)
        os.system("dot -T png graphs/dot/%s/cvp/prune_full_dt_%s.dot -o graphs/%s/cvp/prune_full_dt_%s.png"%(LABEL,measure,LABEL,measure))

        os.makedirs(os.path.dirname('graphs/dot/%s/cvp/%s/best_pruned.dot'%(LABEL,measure)), exist_ok = True)
        os.makedirs(os.path.dirname('graphs/%s/cvp/%s/best_pruned.png'%(LABEL,measure)), exist_ok = True)

        tree.export_graphviz(best_tree, out_file='graphs/dot/%s/cvp/%s/best_pruned.dot'%(LABEL,measure), feature_names= names[:-1],
                        filled=True, rounded=True,special_characters=True)
        os.system("dot -T png graphs/dot/%s/cvp/%s/best_pruned.dot -o graphs/%s/cvp/%s/best_pruned.png"%(LABEL,measure,LABEL,measure))
        #Results after pruning
        if measure == "reg_accuracy":
            func.print_results_class(best_tree, train_raw, test_raw, validation_raw, LABEL, "nach", noten=True)
        else:
            func.print_results_reg(best_tree, train_raw, test_raw, validation_raw, "nach")
        #Store model permanently
        os.makedirs(os.path.dirname("pruned_models/CVP/%s/%s/%s_%s.pkl"%(LABEL,measure,LABEL,measure)), exist_ok = True)
        temp_pkl_file = open("pruned_models/CVP/%s/%s/%s_%s.pkl"%(LABEL,measure,LABEL,measure), "wb")
        pickle.dump(copy.deepcopy(best_tree), temp_pkl_file)
        temp_pkl_file.close()
        

    other_count += 1

====================Prog1_scaled==============
====================reg_accuracy==============
====================mse==============
RMSE vor Pruning Training:  2.0545200048263767e-14
MAE vor Pruning Training:  8.037286683188018e-15
RMSE vor Pruning Test:  168.21278709496772
MAE vor Pruning Test:  130.52726197482693
RMSE vor Pruning Validation:  153.4857960399924
MAE vor Pruning Validation:  130.6356631507868
153.4857960399924
4700
RMSE nach Pruning Training:  10.259134112865159
MAE nach Pruning Training:  1.3135475226244013
RMSE nach Pruning Test:  162.42795750456375
MAE nach Pruning Test:  126.19752342041505
RMSE nach Pruning Validation:  153.4857960399924
MAE nach Pruning Validation:  130.6356631507868
====================mae==============
RMSE vor Pruning Training:  2.0545200048263767e-14
MAE vor Pruning Training:  8.037286683188018e-15
RMSE vor Pruning Test:  168.21278709496772
MAE vor Pruning Test:  130.52726197482693
RMSE vor Pruning Validation:  153.4857960399924
MAE vor Pruning

[[20  0  0  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0]
 [ 1  0  0  8  0  0  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0]
 [ 0  0  0  0  0  0  0 13  0  0  0]
 [ 0  0  0  0  0  0  1  0  6  0  0]
 [ 0  0  0  0  0  0  0  0  0  7  0]
 [ 0  0  0  0  0  1  0  0  0  0 25]]
ACC vor Pruning Training:  0.9763779527559056
[[ 6  1  5  0  0  5  0  2  0  0  1]
 [ 8  0  1  0  1  0  2  1  0  0  1]
 [ 4  1  2  1  0  2  0  0  0  0  2]
 [ 0  1  0  0  0  2  1  4  0  0  1]
 [ 1  1  0  0  1  0  2  1  2  1  1]
 [ 0  0  2  0  0  1  0  2  2  1  1]
 [ 0  0  0  0  0  3  0  1  0  0  1]
 [ 0  0  1  0  0  1  0  0  1  1  3]
 [ 0  1  2  0  0  0  0  0  1  0  0]
 [ 0  1  0  0  0  0  1  3  0  1  1]
 [ 4  2  0  3  3  3  3 16  2  1  4]]
ACC vor Pruning Test:  0.11594202898550725
[[5 0 0 0 0 0 1 0 0 0 0]
 [2 0 1 0 0 0 0 1 0 0 0]
 [0 0 2 0 0 2 0 0 0 0 2]
 [0 0 0 0 0 0 0 3 1 0 0]
 [0 1 0 0 0 0 0 1